In [1]:
import pulp as p
import numpy as np
import math
import pandas as pd

from scipy import stats

import random

import math

<img src = "w4l1.png">
<img src = "w4l2.png">

In [9]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([2800,2800,1000,920,780,950,1050,1200,2000,2500,3000,2800])

materialcost = 75
holdingcost = 25
backordercost = 50 #stockoutcost
hiringcost = 1200
firingcost = 3600
laborhoursperitem = 4
workercost = 2400
overtimecost = 25
outsourcecost = 175
workerhours = 160
maxovertime = 10
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 36
endingworkforcemin = 20
endingworkforcemax = 36


prob = p.LpProblem('Production', p.LpMinimize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, demandqty))


prob += p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod])

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  3308750.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  0.0
Backorder_Qty_3  =  0.0
Backorder_Qty_4  =  0.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  4.0
Firing_2  =  0.0
Firing_3  =  11.0
Firing_4  =  2.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  0.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  1.0
Hiring_8  =  16.0
Hiring_9  =  0.0
In_house_Production_1  =  1440.0
In_house_Production_10  =  1600.0
In_house_Production_11  =  1600.0
In_house_Production_12  =  1440.0
In_house_Production_2  =  1440.0
In_house_Production_3  =  1000.0
In_house_Productio

<img src = "w4l3.png">
<img src = "w4l4.png">

In [8]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([2800,2800,1000,920,780,950,1050,1200,2000,2500,3000,2800])

price = 180
elasticity = 5

discount = np.array([0.1,0.1,0,0,0,0,0,0,0,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 75
holdingcost = 25
backordercost = 50 #stockoutcost
hiringcost = 1200
firingcost = 3600
laborhoursperitem = 4
workercost = 2400
overtimecost = 25
outsourcecost = 175
workerhours = 160
maxovertime = 10
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 36
endingworkforcemin = 20
endingworkforcemax = 36


prob = p.LpProblem('Production Profitability', p.LpMaximize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))


prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  478050.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  0.0
Backorder_Qty_3  =  0.0
Backorder_Qty_4  =  0.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  4.0
Firing_2  =  0.0
Firing_3  =  11.0
Firing_4  =  2.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  0.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  1.0
Hiring_8  =  16.0
Hiring_9  =  0.0
In_house_Production_1  =  1440.0
In_house_Production_10  =  1600.0
In_house_Production_11  =  1600.0
In_house_Production_12  =  1440.0
In_house_Production_2  =  1440.0
In_house_Production_3  =  1000.0
In_house_Production

<img src = "w4l5.png">
<img src = "w4l7.png">
<img src = "w4l6.png">

In [7]:
CDC = ['CDC'] 
ID = ['ID1', 'ID2']
ID_cap = np.array([20000, 5000])
ID_var_cost = np.array([1.35,1.6])
ID_fix_cost = np.array([1100, 900])

POD = ['Conv_Store', 'Retail_Store', 'APS', 'Home/Office']
demandpod = [4500,15000,1000,3000]
podvarcost = [0.5,0,0.75,0]


cost_cdc_id = np.array([[2.4,2.45]])
cost_cdc_pod = np.array([[6,5.85,6.25,8.15]])
                        #pod1 #pod2 #pod3 #pod4
cost_id_pod = np.array([[2.2,2.1,9999999,9999999],#id1
                        [9999999,9999999,1.05,3.4]])#id2

ID_cap_dict = p.makeDict([ID], ID_cap)
ID_var_cost_dict = p.makeDict([ID], ID_var_cost)
ID_fix_cost_dict = p.makeDict([ID], ID_fix_cost)

POD_demand_dict = p.makeDict([POD], demandpod)
POD_var_cost_dict = p.makeDict([POD], podvarcost)

cost_cdc_id_dict = p.makeDict([CDC,ID], cost_cdc_id)
cost_cdc_pod_dict = p.makeDict([CDC,POD], cost_cdc_pod)
cost_id_pod_dict = p.makeDict([ID,POD], cost_id_pod)


prob = p.LpProblem('Distribution', p.LpMinimize)

flow_cdc_id = p.LpVariable.dicts("route", (CDC,ID), lowBound = 0, cat = 'Integer')
flow_cdc_pod = p.LpVariable.dicts("route", (CDC,POD), lowBound = 0, cat = 'Integer')
flow_id_pod = p.LpVariable.dicts("route", (ID,POD), lowBound = 0, cat = 'Integer')

id_open = p.LpVariable.dicts("open", ID, cat = 'Binary')

prob += p.lpSum([flow_cdc_id[c][i]*cost_cdc_id_dict[c][i] for c in CDC for i in ID])\
            + p.lpSum([flow_cdc_pod[c][d]*cost_cdc_pod_dict[c][d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*cost_id_pod_dict[i][d] for i in ID for d in POD])\
            + p.lpSum([id_open[i]*ID_fix_cost_dict[i] for i in ID])\
            + p.lpSum([flow_id_pod[i][d]*ID_var_cost_dict[i] for i in ID for d in POD])\
            + p.lpSum([flow_cdc_pod[c][d]*POD_var_cost_dict[d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*POD_var_cost_dict[d] for i in ID for d in POD])\
                   
for i in ID:
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) == p.lpSum([flow_cdc_id[c][i] for c in CDC])
    
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) <= ID_cap_dict[i]
    
    prob += p.lpSum(flow_cdc_id[c][i] for c in CDC) - 1000000*id_open[i] <= 0
    
for d in POD:
    prob += p.lpSum([flow_id_pod[i][d] for i in ID]) + p.lpSum([flow_cdc_pod[c][d] for c in CDC])  >= POD_demand_dict[d] #demand constraint
    
prob += p.lpSum([id_open[i] for i in ID]) <= 2 #number of ID maximum must be open
                  
prob += p.lpSum([id_open[i] for i in ID]) >= 1 #number of ID minimum must be open

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  146100.0
Solution Status: Optimal
open_ID1  =  0.0
open_ID2  =  1.0
route_CDC_APS  =  0.0
route_CDC_Conv_Store  =  4500.0
route_CDC_Home_Office  =  0.0
route_CDC_ID1  =  0.0
route_CDC_ID2  =  4000.0
route_CDC_Retail_Store  =  15000.0
route_ID1_APS  =  0.0
route_ID1_Conv_Store  =  0.0
route_ID1_Home_Office  =  0.0
route_ID1_Retail_Store  =  0.0
route_ID2_APS  =  1000.0
route_ID2_Conv_Store  =  0.0
route_ID2_Home_Office  =  3000.0
route_ID2_Retail_Store  =  0.0


<img src = "w4l8.png">
<img src = "w4l9.png">

In [6]:
#CDC = Supply Nodes, ID = Intermediate Nodes, POD = Demand Nodes

CDC = ['CP1','CP2','CP3']
CDC_cap = [2000,1500,2500]

ID = ['Regional_Sorting_Depot']
ID_cap = np.array([6000])
ID_var_cost = np.array([5.4])
ID_fix_cost = np.array([700])

POD = ['Recycling Facilities1', 'Recycling Facilities2']
podcap = [3500,3000]
podvarcost = [74.8,83.6]
podfixedcost = [1500,1300]


cost_cdc_id = np.array([[28.4],[26.1],[23.8]])
                         #pod1 #pod2
cost_cdc_pod = np.array([[66.4,56.2], #cdc1
                         [62.4,46.2], #cdc2
                         [73.9,53.6]]) #cdc3
                        #pod1 #pod2
cost_id_pod = np.array([[17.6,19.2]])#id1

CDC_cap_dict = p.makeDict([CDC], CDC_cap) 

ID_cap_dict = p.makeDict([ID], ID_cap)
ID_var_cost_dict = p.makeDict([ID], ID_var_cost)
ID_fix_cost_dict = p.makeDict([ID], ID_fix_cost)

POD_cap_dict = p.makeDict([POD], podcap)
POD_var_cost_dict = p.makeDict([POD], podvarcost)
POD_fix_cost_dict = p.makeDict([POD], podfixedcost)

cost_cdc_id_dict = p.makeDict([CDC,ID], cost_cdc_id)
cost_cdc_pod_dict = p.makeDict([CDC,POD], cost_cdc_pod)
cost_id_pod_dict = p.makeDict([ID,POD], cost_id_pod)


prob = p.LpProblem('Battery_Recycling', p.LpMinimize)

flow_cdc_id = p.LpVariable.dicts("route", (CDC,ID), lowBound = 0, cat = 'Integer')
flow_cdc_pod = p.LpVariable.dicts("route", (CDC,POD), lowBound = 0, cat = 'Integer')
flow_id_pod = p.LpVariable.dicts("route", (ID,POD), lowBound = 0, cat = 'Integer')

id_open = p.LpVariable.dicts("open", ID, cat = 'Binary')
pod_open = p.LpVariable.dicts("open", POD, cat = 'Binary')

prob += p.lpSum([flow_cdc_id[c][i]*cost_cdc_id_dict[c][i] for c in CDC for i in ID])\
            + p.lpSum([flow_cdc_pod[c][d]*cost_cdc_pod_dict[c][d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*cost_id_pod_dict[i][d] for i in ID for d in POD])\
            + p.lpSum([id_open[i]*ID_fix_cost_dict[i] for i in ID])\
            + p.lpSum([flow_id_pod[i][d]*ID_var_cost_dict[i] for i in ID for d in POD])\
            + p.lpSum([flow_cdc_pod[c][d]*POD_var_cost_dict[d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*POD_var_cost_dict[d] for i in ID for d in POD])\
            + p.lpSum([pod_open[d]*POD_fix_cost_dict[d] for d in POD])
                   
for i in ID:
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) == p.lpSum([flow_cdc_id[c][i] for c in CDC])
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) <= ID_cap_dict[i]
    prob += p.lpSum(flow_cdc_id[c][i] for c in CDC) - 1000000000*id_open[i] <= 0
    
for d in POD:
    prob += p.lpSum([flow_id_pod[i][d] for i in ID]) + p.lpSum([flow_cdc_pod[c][d] for c in CDC])  <= POD_cap_dict[d]
    prob += p.lpSum([flow_id_pod[i][d] for i in ID]) + p.lpSum([flow_cdc_pod[c][d] for c in CDC]) - 1000000000*pod_open[d] <= 0 
    
for c in CDC:
    prob += p.lpSum([flow_cdc_id[c][i] for i in ID]) + p.lpSum([flow_cdc_pod[c][d] for d in POD]) == CDC_cap_dict[c]
    
prob += p.lpSum([pod_open[d] for d in POD]) <= 2 #number of POD maximum must be open
prob += p.lpSum([pod_open[d] for d in POD]) >= 1 #number of POD minimum must be open

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  765000.0
Solution Status: Optimal
open_Recycling_Facilities1  =  1.0
open_Recycling_Facilities2  =  1.0
open_Regional_Sorting_Depot  =  1.0
route_CP1_Recycling_Facilities1  =  0.0
route_CP1_Recycling_Facilities2  =  0.0
route_CP1_Regional_Sorting_Depot  =  2000.0
route_CP2_Recycling_Facilities1  =  0.0
route_CP2_Recycling_Facilities2  =  1500.0
route_CP2_Regional_Sorting_Depot  =  0.0
route_CP3_Recycling_Facilities1  =  0.0
route_CP3_Recycling_Facilities2  =  0.0
route_CP3_Regional_Sorting_Depot  =  2500.0
route_Regional_Sorting_Depot_Recycling_Facilities1  =  3500.0
route_Regional_Sorting_Depot_Recycling_Facilities2  =  1000.0


<h3>Practice Problem 1: AB Mower</h3>
    
AB Mower is a small manufacturer of lawn mowers located in southern Sweden. Based on the forecast for the upcoming year, they have created an aggregate production plan available here (Excel) or here (LibreOffice).

Due to labor laws and union agreements, laying off hired workers is not a practical possibility. Owing to this, the seasonal demand for lawn-mowers has traditionally been met by producing ahead of demand during the winter season, and keep these units in stock until demand increases towards the spring.

All costs are reported in Swedish kronor (SEK).

<img src = "w4pp1.png">

In the plan for the coming year, the planner forgot to include the fact that, due to a new deal with the labor union, cost for overtime has increased from SEK 200 per hour to SEK 220 per hour. The planner realizes this will increase costs substantially, and like to know if they should plan to hire workers to account for this change.

Part2

Please complete the model and run it to minimize cost.

How many workers should AB Mower hire to minimize cost for the next year?

Round your answer to the nearest integer

Part3

When the planner discusses the plan with the logistics manager, the manager realizes that the plan is based on a desired service level (fill rate) of around 86%. Month 7 has a service level well below 90%. He believes this is too low, and asks you to increase the cost of stockout to SEK 1,900 per unit and month. Overtime costs are still SEK 220.

Please run the model again with the new parameter values.

What is the forecasted fill rate in Month 8 with the new optimal plan?

In [5]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([2800,2800,2900,4000,4500,4200,4100,3300,3100,2600,2400,2800])

price = 6000
elasticity = 4

discount = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 3000
holdingcost = 100
backordercost = 1900 #stockout cost
hiringcost = 5000
firingcost = 100000
laborhoursperitem = 10
workercost = 24000
overtimecost = 220
outsourcecost = 5500
workerhours = 160
maxovertime = 20
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 200
endingworkforcemin = 170
endingworkforcemax = 230

#prob = p.LpProblem('Production Cost', p.LpMaximize)
prob = p.LpProblem('Production Cost', p.LpMinimize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

prob += p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod])

#prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
#        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
#        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
#        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
#        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  182400000.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  0.0
Backorder_Qty_3  =  0.0
Backorder_Qty_4  =  0.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  100.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  0.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  0.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  0.0
In_house_Production_1  =  3200.0
In_house_Production_10  =  2600.0
In_house_Production_11  =  2500.0
In_house_Production_12  =  3200.0
In_house_Production_2  =  3200.0
In_house_Production_3  =  3200.0
In_house_Product

Part 4

The planner now wants to include the sales planning into the aggregate model. He gets the sales manager involved in the process and learns that the sales team was thinking about launching an end-of-season promotion to boost sales in Month 9. Initially, they were thinking about reducing the price of a lawn mower by 10% during this month. The assumptions from Part 3 still apply.

Would it be profitable to reduce prices by 10% in month 9, assuming a price elasticity of demand of -4?

In [19]:

timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([2800,2800,2900,4000,4500,4200,4100,3300,3100,2600,2400,2800])

price = 6000
elasticity = 4

discount = np.array([0,0,0,0,0,0,0,0,0.1,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 3000
holdingcost = 100
backordercost = 1900 #stockout cost
hiringcost = 5000
firingcost = 100000
laborhoursperitem = 10
workercost = 24000
overtimecost = 220
outsourcecost = 5500
workerhours = 160
maxovertime = 20
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 200
endingworkforcemin = 170
endingworkforcemax = 230


prob = p.LpProblem('Production Cost', p.LpMaximize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    
    
    
prob.solve()

print('Revenue ', p.lpSum([demands[t]*prices[t] for t in timeperiod]))
print('Cost', p.lpSum([demands[t]*prices[t] for t in timeperiod]) - p.value(prob.objective))

print('Optimal Profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

Revenue  241836000.0
Cost 188090400.0
Optimal Profit:  53745600.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  0.0
Backorder_Qty_3  =  0.0
Backorder_Qty_4  =  0.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  792.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  0.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  14.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  0.0
In_house_Production_1  =  3424.0
In_house_Production_10  =  3392.0
In_house_Production_11  =  2400.0
In_house_Production_12  =  3300.0
In_house_Production_2  =  3424.0
In_house_

<h3>Practice Problem 2: Dire Wolf Revisited</h3>

You are working for Dire Wolf Industries (DWI) helping them run their Sales & Operations Planning (S&OP) program for a line of make to stock items. Specifically, you are running the aggregate planning model to help assist the Chief Operating Officer (COO) and the Vice President of Sales to agree upon the next 12 months plan.  

<img src = "w4pp21.png">

<b>Part 1</b>

You want to develop an initial aggregate plan for the next 12 months. Use the initial data provided in the spreadsheet and assume that there are no planned promotions or discounts over the next 12 months. Find the optimal plan over the next 12 months and answer the following questions.

What is the number of employees hired over the next 12 months?

<b>Part 2</b>

You have been asked to see if any of the following recommendations from your Chief Operating Officer (COO) will lead to a better solution. Treat each strategy independently.

Which of the following changes will lead to an increase in total annual profit according to the aggregate plan?


Select all correct answers

Increase the allowable overtime hours from 10 to 20 hours per employee per month.

Do not allow any backlogging of orders.

Keep the starting workforce at 24 people, but allow the model to hire up to 6 more people over the course of the year.

Negotiate with your employees to increase the number of regular hours worked per month from 160 to 180 and increase the monthly wages from 2,400 to 2800 per employee per month.

None of these strategies increases the total annual profit.

<b>Part 3</b>

None of the strategies in Part 2 were taken. The COO decides to use the plan described in Part 1. However, the Vice Presidents of Sales and of Marketing have stated that they think a promotion would help increase total profits.

They believe that every 1 percent decrease in the base price will increase the sales by 5 percent. That is, they assume an elasticity of demand with respect to price to be -5. Remember, in the spreadsheet we enter this in as a positive number since the percent price discount is a positive number.

They want to run a promotion for one and only one month during the next 12 months where the price is discounted 20%.

Which month should the promotion be run in order to maximize the total annual profit?

<b>Part 4</b>

The promotion in Part 3 was not run. Instead, go back to the model from Part 1. The senior team has decided to run a one month promotion during September (month 9) at 25% since this is when most of their customers decide to make their purchases.

They believe that every 1 percent decrease in the base price will increase the sales by 6 percent. That is, they assume an elasticity of demand with respect to price to be -6. Remember, in the spreadsheet we enter this in as a positive number since the percent price discount is a positive number.

Which of the following statements are true if this promotion is planned?


Select all correct answers

The profitability (as compared to Part 1) will increase

The profitability (as compared to Part 1) will decrease

The total units sold (as compared to Part 1) will increase

The total units sold (as compared to Part 1) will decrease

The total revenue in dollars (as compared to Part 1) will increase

The total revenue in dollars (as compared to Part 1) will decrease

In [22]:
#PRODUCTION PROFIT MAXIMIZATION

timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([3000,2800,1000,600,400,550,800,1000,1200,1400,3000,3200])

price = 210
elasticity = 6

discount = np.array([0,0,0,0,0,0,0,0,0.25,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 80
holdingcost = 15
backordercost = 30 #stockout cost
hiringcost = 2400
firingcost = 12000
laborhoursperitem = 3
workercost = 2400
overtimecost = 20
outsourcecost = 180
workerhours = 160
maxovertime = 10
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 24
endingworkforcemin = 24
endingworkforcemax = 24


prob = p.LpProblem('Optimal_Production', p.LpMaximize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    

prob.solve()

print('Revenue ', p.lpSum([demands[t]*prices[t] for t in timeperiod]))
print('Cost', p.lpSum([demands[t]*prices[t] for t in timeperiod]) - p.value(prob.objective))
print('Total Unit Sold', newdemand.sum())

print('Optimal_Profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

Revenue  4200000.0
Cost 3052450.0
Total Unit Sold 20750.0
Optimal_Profit:  1147550.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  1440.0
Backorder_Qty_3  =  1080.0
Backorder_Qty_4  =  400.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  0.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  0.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  0.0
In_house_Production_1  =  1360.0
In_house_Production_10  =  1360.0
In_house_Production_11  =  1360.0
In_house_Production_12  =  1360.0
In_house_Productio

<b>Part 5</b>

Again, the senior leadership team has decided to not pursue any of the promotion strategies. You are still using the base model from Part 1. You become aware of the fact that your plant in San Francisco has very limited storage space. In fact, you learn that you cannot store more than 1000 units in any one month during the next 12 months.

Develop an aggregate plan that enforces this new constraint and answer the following questions.

How much lower is your total annual profit with this new plan compared to Part 1? Enter your answer as an integer.

In [25]:
#PRODUCTION PROFIT MAXIMIZATION

timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([3000,2800,1000,600,400,550,800,1000,1200,1400,3000,3200])

price = 210
elasticity = 6

discount = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 80
holdingcost = 15
backordercost = 30 #stockout cost
hiringcost = 2400
firingcost = 12000
laborhoursperitem = 3
workercost = 2400
overtimecost = 20
outsourcecost = 180
workerhours = 160
maxovertime = 10
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 24
endingworkforcemin = 24
endingworkforcemax = 24

storagecap = 1000 #maximum storage capacity constraint


prob = p.LpProblem('Optimal_Production', p.LpMaximize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    
    prob += inventory[t] <= storagecap #max storage constraint
    
prob.solve()

print('Revenue ', p.lpSum([demands[t]*prices[t] for t in timeperiod]))
print('Cost', p.lpSum([demands[t]*prices[t] for t in timeperiod]) - p.value(prob.objective))
print('Total Unit Sold', newdemand.sum())

print('Optimal_Profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

Revenue  3979500
Cost 2802300.0
Total Unit Sold 18950
Optimal_Profit:  1177200.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  1440.0
Backorder_Qty_3  =  1080.0
Backorder_Qty_4  =  400.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  0.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  0.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  0.0
In_house_Production_1  =  1360.0
In_house_Production_10  =  1360.0
In_house_Production_11  =  1360.0
In_house_Production_12  =  1360.0
In_house_Production_2 

<h3>Practice Problem 3: Araz Revisited</h3>
    
For this problem, we will use the Model for Araz. To recap, Araz is a fast-fashion retailer selling mainly to fashion conscious women on a budget. It currently distributes its products from its own plants to its own branded stores or directly from fulfillment centers to home for online sales. It has decided to offer different delivery options for its online (e- commerce & m-commerce) consumers:

Home delivery

Pick up in their stores

Pick up in a convenience store (e.g. an attended pick up point)

Pick up in an Automated Package Station (e.g. “un-attended” pick up point)

Araz stores its inventory of goods at a central distribution center (CDC) which is used to send goods through all the above channels. Some of the channels get the goods directly from the CDC but it also has two intermediary depots to reach all the channels on time and also keep up with the demand. The network design depicting the nodes and possible flows was shown in the video lecture and can be referred from the slides. You have the following data provided to you about the capacities of depots, costs and quantities demanded at the various channels.

The capacity at the CDC is unlimited, while the capacity at the Intermediary Depot 1 is 20000 boxes/month and Intermediary DC 2 has a capacity of 5000 boxes/month. You can assume there is no cost associated with operating the CDC, as this will not influence the choice of channels and flows.

<img src = "w4l6.png">

<b>Part 1</b>

Over a period of time, Araz notices that the demand from its self-serve customers is increasing and it updates its estimates of quantity demanded through the channels APS and Home to 2000 and 4000 boxes/month respectively. Araz is willing to use anywhere between 0 to 2 depots in order to serve its customers.

What is the new optimal total cost?

What is the quantity of goods flowing directly from the CDC to customers Home?

How much spare capacity is left unused at Intermediary Depot 2?

<b>Part 2</b>

Noticing the limitations, Araz decides to double the capacity of its Intermediary Depot 2 to 10000 boxes/month.

What is the new optimal total cost?

What is the quantity of goods flowing directly from the CDC to customers Home?

How much spare capacity is left unused at Intermediary Depot 2?

<b>Part 3</b>

The freight contract of transporter moving goods from the CDC got revised this year. The costs of transporting goods from CDC have gone up by approximately 1%. The revised POD transportations costs from CDC are as follows:

CDC to Convenience Store = 6.06 $/box

CDC to Retail Store = 5.95 $/box

CDC to APS = 6.31 $/box

CDC to Home = $8.23 $/box

What is the new optimal total cost?

How many Intermediary Depots are open? Options = 0, 1, 2

What is the total quantity of goods flowing directly from the CDC to all Channels?

In [32]:
CDC = ['CDC'] 
ID = ['ID1', 'ID2']
ID_cap = np.array([20000, 10000])
ID_var_cost = np.array([1.35,1.6])
ID_fix_cost = np.array([1100, 900])

POD = ['Conv_Store', 'Retail_Store', 'APS', 'Home/Office']
demandpod = [4500,15000,2000,4000]
podvarcost = [0.5,0,0.75,0]


cost_cdc_id = np.array([[2.4,2.45]])
cost_cdc_pod = np.array([[6.06,5.95,6.31,8.23]])
                        #pod1 #pod2 #pod3 #pod4
cost_id_pod = np.array([[2.2,2.1,9999999,9999999],#id1
                        [9999999,9999999,1.05,3.4]])#id2

ID_cap_dict = p.makeDict([ID], ID_cap)
ID_var_cost_dict = p.makeDict([ID], ID_var_cost)
ID_fix_cost_dict = p.makeDict([ID], ID_fix_cost)

POD_demand_dict = p.makeDict([POD], demandpod)
POD_var_cost_dict = p.makeDict([POD], podvarcost)

cost_cdc_id_dict = p.makeDict([CDC,ID], cost_cdc_id)
cost_cdc_pod_dict = p.makeDict([CDC,POD], cost_cdc_pod)
cost_id_pod_dict = p.makeDict([ID,POD], cost_id_pod)


prob = p.LpProblem('Distribution', p.LpMinimize)

flow_cdc_id = p.LpVariable.dicts("route", (CDC,ID), lowBound = 0, cat = 'Integer')
flow_cdc_pod = p.LpVariable.dicts("route", (CDC,POD), lowBound = 0, cat = 'Integer')
flow_id_pod = p.LpVariable.dicts("route", (ID,POD), lowBound = 0, cat = 'Integer')

id_open = p.LpVariable.dicts("open", ID, cat = 'Binary')

prob += p.lpSum([flow_cdc_id[c][i]*cost_cdc_id_dict[c][i] for c in CDC for i in ID])\
            + p.lpSum([flow_cdc_pod[c][d]*cost_cdc_pod_dict[c][d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*cost_id_pod_dict[i][d] for i in ID for d in POD])\
            + p.lpSum([id_open[i]*ID_fix_cost_dict[i] for i in ID])\
            + p.lpSum([flow_id_pod[i][d]*ID_var_cost_dict[i] for i in ID for d in POD])\
            + p.lpSum([flow_cdc_pod[c][d]*POD_var_cost_dict[d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*POD_var_cost_dict[d] for i in ID for d in POD])\
                   
for i in ID:
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) == p.lpSum([flow_cdc_id[c][i] for c in CDC])
    
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) <= ID_cap_dict[i]
    
    prob += p.lpSum(flow_cdc_id[c][i] for c in CDC) - 1000000*id_open[i] <= 0
    
for d in POD:
    prob += p.lpSum([flow_id_pod[i][d] for i in ID]) + p.lpSum([flow_cdc_pod[c][d] for c in CDC])  >= POD_demand_dict[d] #demand constraint
    
prob += p.lpSum([id_open[i] for i in ID]) <= 2 #number of ID maximum must be open
                  
prob += p.lpSum([id_open[i] for i in ID]) >= 0 #number of ID minimum must be open

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  160275.0
Solution Status: Optimal
open_ID1  =  1.0
open_ID2  =  1.0
route_CDC_APS  =  0.0
route_CDC_Conv_Store  =  0.0
route_CDC_Home_Office  =  0.0
route_CDC_ID1  =  19500.0
route_CDC_ID2  =  6000.0
route_CDC_Retail_Store  =  0.0
route_ID1_APS  =  0.0
route_ID1_Conv_Store  =  4500.0
route_ID1_Home_Office  =  0.0
route_ID1_Retail_Store  =  15000.0
route_ID2_APS  =  2000.0
route_ID2_Conv_Store  =  0.0
route_ID2_Home_Office  =  4000.0
route_ID2_Retail_Store  =  0.0


<h2>Graded Assignment 1: Dire Wolf Redux</h2>
    
You are working for Dire Wolf Industries (DWI), helping them run their Sales & Operations Planning (S&OP) program for a line of make to stock items. Specifically, you are running the aggregate planning model to help assist the Chief Operating Officer (COO) and the Vice President of Sales to agree upon the next 12 months plan.

The demand data for the model is provided below, and you must enter it into the spreadsheet. The aggregate monthly demand of the item is as follows:

January	February	March	April
13835	12515	11345	8665
May	June	July	August
5045	2615	1465	3665
September	October	November	December
6385	7555	9935	15335

The remaining data for the model is provided below, and you must enter it into the spreadsheet. Please enter the input values with all the decimal values given below.

Material cost ($/unit): 12.166

Inventory holding costs ($/unit/month): 2.618

Cost of stockouts ($/unit/month): 4.774

Cost for hiring and training new worker: 1925.0

Cost for laying off an employee: 3850.0

Labor hours required per item produced: 0.66

Regular worker cost ($/month): 1848.0

Overtime cost ($/hour): 13.3

Outsourcing cost ($/item): 27.874

Hours worked by employee per month: 172.7

Max overtime (hours/month/employee): 12.1

Starting inventory: 2500

Ending inventory (min): 2500

Starting backlog: 0

Allowable ending backlog (max): 0

Starting workforce: 24

Ending workforce (min): 24

Ending workforce (max): 24

Base price ($/item): 32.494

<b>Part 1</b>

You want to develop an initial aggregate plan for the next 12 months. Use the initial data provided above and assume that there are no planned promotions or discounts over the next 12 months. Find the optimal plan over the next 12 months. Based on this optimal solution, answer the following questions.

<b>Part 2</b>

Using the status quo (Part 1) as starting point, the senior team has asked you to assess the impact of running a three-month promotion during the Summer with a 15% discount in June (month 6), a 25% discount in July (month 7), and a 15% discount in August (month 8). Use an elasticity value of 5, and solve optimally.

Will the total annual profit increase in this scenario, compared to the profit in Part 1?

<b>Part 3</b>

There are some concerns about the sustainability of the operations at the subcontractors. Using the status quo (Part 1) as starting point, the senior team has asked you to assess the impact of a policy that would eliminate all outsourcing of production. Solve optimally.

How will the total annual profit under this policy compare to the profit of the status quo (Part 1)?

<b>Part 4</b>

There is some new technology coming into the market that may allow Dire Wolf to produce the items faster than before. Using the status quo (Part 1) as starting point, you have been asked to assess the impact that such a technology could have if it could reduce the labor hours required per item produced by one third. (Hint: This means that the hours required to produce an item using the new technology would be two thirds of the hours required to produce an item in the status quo). Solve optimally.

Which of the following statements would be true considering the faster production scenario, compared to the status quo (Part 1)?

Select all the correct answers. 

Number of employees fired decreases by one third

Total annual profit increases

Outsourced production is no longer used

Firing employees is no longer done

None of the above

Solving optimally, what is the total annual profit in this scenario?

In [12]:
#PRODUCTION PROFIT MAXIMIZATION

timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([13835,12515,11345,8665,5045,2615,1465,3665,6385,7555,9935,15335])

price = 32.494
elasticity = 5

discount = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 12.166
holdingcost = 2.618
backordercost = 4.774 #stockout cost
hiringcost = 1925
firingcost = 3850
laborhoursperitem = 0.66
workercost = 1848
overtimecost = 13.3
outsourcecost = 27.874
workerhours = 172.7
maxovertime = 12.1
startinginventory = 2500
endinginventorymin = 2500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 24
endingworkforcemin = 24
endingworkforcemax = 24


prob = p.LpProblem('Optimal_Production', p.LpMaximize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Integer')

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Integer')

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    

prob.solve()

print('Revenue ', p.lpSum([demands[t]*prices[t] for t in timeperiod]))
print('Cost', p.lpSum([demands[t]*prices[t] for t in timeperiod]) - p.value(prob.objective))
print('Total Unit Sold', newdemand.sum())

print('Optimal_Profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

Revenue  3196109.84
Cost 2185893.4020000007
Total Unit Sold 98360
Optimal_Profit:  1010216.437999999
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  14.0
Backorder_Qty_3  =  159.0
Backorder_Qty_4  =  1235.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  11.0
Firing_5  =  5.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  16.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  0.0
In_house_Production_1  =  11200.0
In_house_Production_10  =  6720.0
In_house_Production_11  =  6720.0
In_house_Production_12  =  6720.0
I

<b>Part 5</b>

There are some concerns about the limitations of Dire Wolf's warehousing capacity. Using the status quo (Part 1) as starting point, the senior team has asked you to assess the impact of limiting inventory to a maximum of 5000 items. (Hint: This means that at no point can the inventory levels exceed this limit). Solve optimally.

What is the total annual profit?

In [13]:
#PRODUCTION PROFIT MAXIMIZATION

timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([13835,12515,11345,8665,5045,2615,1465,3665,6385,7555,9935,15335])

price = 32.494
elasticity = 5

discount = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 12.166
holdingcost = 2.618
backordercost = 4.774 #stockout cost
hiringcost = 1925
firingcost = 3850
laborhoursperitem = 0.66
workercost = 1848
overtimecost = 13.3
outsourcecost = 27.874
workerhours = 172.7
maxovertime = 12.1
startinginventory = 2500
endinginventorymin = 2500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 24
endingworkforcemin = 24
endingworkforcemax = 24
storagecap = 5000


prob = p.LpProblem('Optimal_Production', p.LpMaximize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Integer')

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Integer')

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
  
    prob += inventory[t] <= storagecap #max storage constraint

prob.solve()

print('Revenue ', p.lpSum([demands[t]*prices[t] for t in timeperiod]))
print('Cost', p.lpSum([demands[t]*prices[t] for t in timeperiod]) - p.value(prob.objective))
print('Total Unit Sold', newdemand.sum())

print('Optimal_Profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

Revenue  3196109.84
Cost 2190922.1180000002
Total Unit Sold 98360
Optimal_Profit:  1005187.7219999996
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  13.0
Backorder_Qty_3  =  158.0
Backorder_Qty_4  =  188.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  7.0
Firing_5  =  13.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  16.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  0.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  4.0
In_house_Production_1  =  11200.0
In_house_Production_10  =  6720.0
In_house_Production_11  =  6720.0
In_house_Production_12  =  6720.0
I

<h3>Graded Assignment 2: Fresh Food</h3>
    
Fresh Food is an omnichannel grocery retailer with stores in Phoenix, Arizona. The company wants to design its online grocery business to allow customers two fulfillment options:

A.   Click-and-Collect (CC): To pick up their online orders from a store, or 

B.   Home Delivery (HD): Have them delivered to their home address.

To fulfill these two CC and HD orders, Fresh Foods has four alternatives: 

A.   Pickup: Ship online orders to a store where customers pick them up, 

B.   Direct Delivery: Deliver orders directly from a regional DC (RDC) close to the Phoenix market using company’s fleet of trucks, 

C.    Delivery from In-Market DC (IMDC): Build an intermediary DC inside the Phoenix market. And, ship orders from RDC to IMDC, and then to customer homes,

D.   Outsourced Delivery: Have a third-party deliver orders from the RDC to customer homes. 

The challenge is which channel to utilize for which type of order, given that each option has different costs and capacity limitations.

Monthly demand: The company estimates that it will have 600 Click-and-Collect orders and 800 Home Delivery orders. Note we must fulfill all demand.

Facility cost: The cost of opening and maintaining the IMDC is 1000dollars per month, irrespective of the quantity of orders processed.

<b>Part 1</b>

Capacity: IMDC, if built, can process 100 orders per month. The RDC and the 3rd party provider do not have a capacity limit.
Transportation costs (dollar per order) between facilities and to customers are given below:

<img src = "w4ga2.png">

What is the optimal monthly total cost?

<b>Part 2</b>

Now assume the IMDC has a capacity of 1000 orders per month. What is the optimal monthly total cost?

In [26]:
CDC = ['RDC'] 
ID = ['IMDC']
ID_cap = np.array([1000])
#ID_var_cost = np.array([1.35,1.6])
ID_fix_cost = np.array([1000])

POD = ['CnC/RetailStore', 'Home(Own Delivery)', 'Home(3rd Party Delivery)']
demandpod = [600,0,0]
#podvarcost = [0.5,0,0.75,0]


cost_cdc_id = np.array([[2]])
cost_cdc_pod = np.array([[1,10,13]])
                        #pod1 #pod2 #pod3
cost_id_pod = np.array([[99999999,4,9999999]])#id1

ID_cap_dict = p.makeDict([ID], ID_cap)
#ID_var_cost_dict = p.makeDict([ID], ID_var_cost)
ID_fix_cost_dict = p.makeDict([ID], ID_fix_cost)

POD_demand_dict = p.makeDict([POD], demandpod)
#POD_var_cost_dict = p.makeDict([POD], podvarcost)

cost_cdc_id_dict = p.makeDict([CDC,ID], cost_cdc_id)
cost_cdc_pod_dict = p.makeDict([CDC,POD], cost_cdc_pod)
cost_id_pod_dict = p.makeDict([ID,POD], cost_id_pod)


prob = p.LpProblem('Distribution', p.LpMinimize)

flow_cdc_id = p.LpVariable.dicts("route", (CDC,ID), lowBound = 0, cat = 'Integer')
flow_cdc_pod = p.LpVariable.dicts("route", (CDC,POD), lowBound = 0, cat = 'Integer')
flow_id_pod = p.LpVariable.dicts("route", (ID,POD), lowBound = 0, cat = 'Integer')

id_open = p.LpVariable.dicts("open", ID, cat = 'Binary')

prob += p.lpSum([flow_cdc_id[c][i]*cost_cdc_id_dict[c][i] for c in CDC for i in ID])\
            + p.lpSum([flow_cdc_pod[c][d]*cost_cdc_pod_dict[c][d] for c in CDC for d in POD])\
            + p.lpSum([flow_id_pod[i][d]*cost_id_pod_dict[i][d] for i in ID for d in POD])\
            + p.lpSum([id_open[i]*ID_fix_cost_dict[i] for i in ID])\
            #+ p.lpSum([flow_id_pod[i][d]*ID_var_cost_dict[i] for i in ID for d in POD])\
            #+ p.lpSum([flow_cdc_pod[c][d]*POD_var_cost_dict[d] for c in CDC for d in POD])\
            #+ p.lpSum([flow_id_pod[i][d]*POD_var_cost_dict[d] for i in ID for d in POD])\
                   
for i in ID:
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) == p.lpSum([flow_cdc_id[c][i] for c in CDC])
    
    prob += p.lpSum([flow_id_pod[i][d] for d in POD]) <= ID_cap_dict[i]
    
    prob += p.lpSum(flow_cdc_id[c][i] for c in CDC) - 10000000*id_open[i] <= 0
    
for d in POD:
    prob += p.lpSum([flow_id_pod[i][d] for i in ID]) + p.lpSum([flow_cdc_pod[c][d] for c in CDC])  >= POD_demand_dict[d] #demand constraint

#total demand for home(own delivery) + home(3rd party delivery) must be greater than 800
prob += p.lpSum([flow_id_pod[i]['Home(Own Delivery)'] for i in ID]) + p.lpSum([flow_id_pod[i]['Home(3rd Party Delivery)'] for i in ID])\
        + p.lpSum([flow_cdc_pod[c]['Home(Own Delivery)'] for c in CDC]) + p.lpSum([flow_cdc_pod[c]['Home(3rd Party Delivery)'] for c in CDC]) >= 800
    
prob += p.lpSum([id_open[i] for i in ID]) <= 1 #number of ID maximum must be open
                  
prob += p.lpSum([id_open[i] for i in ID]) >= 0 #number of ID minimum must be open

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  6400.0
Solution Status: Optimal
open_IMDC  =  1.0
route_IMDC_CnC_RetailStore  =  0.0
route_IMDC_Home(3rd_Party_Delivery)  =  0.0
route_IMDC_Home(Own_Delivery)  =  800.0
route_RDC_CnC_RetailStore  =  600.0
route_RDC_Home(3rd_Party_Delivery)  =  0.0
route_RDC_Home(Own_Delivery)  =  0.0
route_RDC_IMDC  =  800.0


<h3>MICROMASTERS PRACTICE PROBLEM 1: AB MOWER REVISITED</h3>
    
Consider again AB Mower from this week's Practice Problem 1. After having met with both the logistics manager and the sales manager, the planner has some further analyses she likes to make.

We pick up where we left AB Mower: overtime costs are SEK 220 per hour.

<b>Part 1</b>

Your last analysis showed that the sales manager's suggestion to run a promotion in Month 9, at a 10% discount, was not profitable. The sales manager has told the planner that, respectfully, he disagrees. The sales team has run the numbers he says, and finds that it is, indeed, profitable. He shows you and the planner using your model: by changing the price in Month 9 to 5,400 the corresponding demand increases to 4,340 - for an increase in profit by more than SEK 7,000,000.

What is the problem with this solution?

answer: In this question, what sales do is just apply 10% discount in month 9 without running the solver. So, you will see that although profit increases after manipulating the model, there are negative inventory in the inventory column which are not included into the calculation of cost of stock out, making the profitability inflate. However, after you run the solver, the negative inventory will convert to backlogged as the model is set up based on the logic that inventory is non-negative.

<b>Part 2</b>

The sales manager buys your argument. He still thinks you should run a promotion at the end of the season to get the last demand before the fall comes.

If AB Mower was to launch a promotion in Month 9, what is the optimal discount? Answer in full percent, without the percentage symbol.

Explanation

There is no simple way to include this in the model without running into non-linearity problems. We could however do a quick check by running the model for a few different values. Pretty quickly we see that the profit is maximized at a 2% discount -- not a whole lot of discount.

<b>Part 3</b>

As the new year is approaching, you hear from one of the purchasers that a new outsourcing option has appeared, that could produce the lawn mower at only SEK 5,000 per unit.

Suppose the promotion from Part 2 will run in Month 9, what is the optimal amount to outsource, in total?

<b>Part 4</b>

Before finalizing the plan, the planner tells you that he has heard from engineering that the plant needs to be down one of the months next year for scheduled maintenance. The outsourcing option from Part 3 still applies, as well as the promotion from Part 2.

When is the best month for scheduled maintenance? Answer with the month number.

In [42]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([2800,2800,2900,4000,4500,4200,4100,3300,3100,2600,2400,2800])

price = 6000
elasticity = 4

discount = np.array([0,0,0,0,0,0,0,0,0.02,0,0,0]) #part2
normalprice = np.array([price]*len(timeperiod))

discountprice = normalprice * (1-discount)
newdemand = np.array(demandqty * (1 + elasticity*discount))
 

materialcost = 3000
holdingcost = 100
backordercost = 1900 #stockout cost
hiringcost = 5000
firingcost = 100000
laborhoursperitem = 10
workercost = 24000
overtimecost = 220
outsourcecost = 5000 #part3
workerhours = 160
maxovertime = 20
startinginventory = 500
endinginventorymin = 500
startingbacklog = 0
endingbacklogmax = 0
startingworkforce = 200
endingworkforcemin = 170
endingworkforcemax = 230

prob = p.LpProblem('Production Cost', p.LpMaximize)
#prob = p.LpProblem('Production Cost', p.LpMinimize)

workers = p.LpVariable.dicts("Workers", timeperiod_with_0, lowBound = 0, cat = 'Integer') #humans must be integer

overtimes = p.LpVariable.dicts("Overtime", timeperiod, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

hirings = p.LpVariable.dicts("Hiring", timeperiod, lowBound = 0, cat = 'Integer') #humans must be integer

firings = p.LpVariable.dicts("Firing", timeperiod, lowBound = 0, cat = 'Integer') ##humans must be integer

inventory = p.LpVariable.dicts("Inventory_on_Hand", timeperiod_with_0, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

backorders = p.LpVariable.dicts("Backorder_Qty", timeperiod_with_0, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

internalprod = p.LpVariable.dicts("In_house_Production", timeperiod, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

outsourceprod = p.LpVariable.dicts("Outsource_Production", timeperiod, lowBound = 0, cat = 'Integer') #choose continuous so as to not enforce integer constraint

internalprod_open = p.LpVariable.dicts("In_house_Production_Open", timeperiod, lowBound = 0, cat = 'Binary') #PART4

demands = dict(zip(timeperiod, newdemand))
prices = dict(zip(timeperiod, discountprice))

#prob += p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
#        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
#        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
#        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod])

prob += p.lpSum([demands[t]*prices[t] for t in timeperiod])\
        - (p.lpSum([workers[t]*workercost for t in timeperiod]) + p.lpSum([inventory[t]*holdingcost for t in timeperiod])\
        + p.lpSum([overtimes[t]*overtimecost for t in timeperiod]) + p.lpSum([hirings[t]*hiringcost for t in timeperiod])\
        + p.lpSum([firings[t]*firingcost for t in timeperiod]) + p.lpSum([backorders[t]*backordercost for t in timeperiod])\
        + p.lpSum([internalprod[t]*materialcost for t in timeperiod]) + p.lpSum([outsourceprod[t]*outsourcecost for t in timeperiod]))

prob += inventory[0] == startinginventory

prob += inventory[timeperiod[-1]] >= endinginventorymin

prob += backorders[0] == startingbacklog

prob += backorders[timeperiod[-1]] <= endingbacklogmax

prob += workers[0] == startingworkforce 

prob += workers[timeperiod[-1]] >= endingworkforcemin

prob += workers[timeperiod[-1]] <= endingworkforcemax

prob += p.lpSum([internalprod_open[t] for t in timeperiod]) == len(timeperiod) - 1 #PART4

for t in timeperiod:
    
    prob += internalprod[t] <= (workerhours*workers[t] + overtimes[t])/laborhoursperitem #max production cap constraint
    
    prob += workers[t] == workers[t-1] + hirings[t] - firings[t] #conservation of human flow
    
    prob += overtimes[t] <= workers[t]*maxovertime #max overtime capacity
    
    prob += inventory[t] == inventory[t-1] + internalprod[t] + outsourceprod[t] + backorders[t] - demands[t] - backorders[t-1] #conservation of inventory flow
    
    prob += internalprod[t] - sum(newdemand)*internalprod_open[t] <= 0 #PART4
    #IMPORTANT: USE THE TOTAL OF THE DEMANDS AS THE BIG NUMBER, NOT A NUMBER THAT IS TOO BIG, 
    #OTHERWISE THE PROGRAM WON'T FIND OPTIMAL SOLUTION 
    
    
prob.solve()

print('Revenue ', p.lpSum([demands[t]*prices[t] for t in timeperiod]))
print('Cost', p.lpSum([demands[t]*prices[t] for t in timeperiod]) - p.value(prob.objective))

print('Optimal Profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

print('Total Outsource Qty: ', p.lpSum([outsourceprod[t].varValue for t in timeperiod])) #part3

Revenue  238086240.0
Cost 186035600.0
Optimal Profit:  52050640.0
Solution Status: Optimal
Backorder_Qty_0  =  0.0
Backorder_Qty_1  =  0.0
Backorder_Qty_10  =  0.0
Backorder_Qty_11  =  0.0
Backorder_Qty_12  =  0.0
Backorder_Qty_2  =  0.0
Backorder_Qty_3  =  0.0
Backorder_Qty_4  =  0.0
Backorder_Qty_5  =  0.0
Backorder_Qty_6  =  0.0
Backorder_Qty_7  =  0.0
Backorder_Qty_8  =  0.0
Backorder_Qty_9  =  0.0
Firing_1  =  0.0
Firing_10  =  0.0
Firing_11  =  0.0
Firing_12  =  0.0
Firing_2  =  0.0
Firing_3  =  0.0
Firing_4  =  0.0
Firing_5  =  0.0
Firing_6  =  0.0
Firing_7  =  0.0
Firing_8  =  0.0
Firing_9  =  0.0
Hiring_1  =  0.0
Hiring_10  =  0.0
Hiring_11  =  0.0
Hiring_12  =  0.0
Hiring_2  =  0.0
Hiring_3  =  30.0
Hiring_4  =  0.0
Hiring_5  =  0.0
Hiring_6  =  0.0
Hiring_7  =  0.0
Hiring_8  =  0.0
Hiring_9  =  0.0
In_house_Production_1  =  3200.0
In_house_Production_10  =  3680.0
In_house_Production_11  =  3680.0
In_house_Production_12  =  0.0
In_house_Production_2  =  3200.0
In_house_Produ